In [1]:
from functools import partial
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
import matplotlib.pyplot as plt

In [2]:
# Load in the data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
(X_train, y_train) , (X_valid, y_valid)   = (X_train_full[:-5000], y_train_full[:-5000]), (X_train_full[-5000:], y_train_full[-5000:])

In [3]:
size = (64, 64)
resize_lambda = lambda image: tf.image.resize(image, size)
X_train = resize_lambda(X_train)
X_valid = resize_lambda(X_valid)
X_test = resize_lambda(X_test)
X_train.shape[1:]

TensorShape([64, 64, 3])

In [4]:
BATCH_SIZE = 100
IMAGE_COUNT = len(X_train)
STEPS_PER_EPOCH = np.ceil(IMAGE_COUNT/BATCH_SIZE)

IMAGE_COUNT, STEPS_PER_EPOCH

(45000, 450.0)

In [5]:
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(CLASS_NAMES)

In [6]:
train_datagen = ImageDataGenerator(
    # rescale= 1./255, 
    # rotation_range=40, 
    # width_shift_range=0.2, 
    # height_shift_range= 0.2, 
    # shear_range= 0.2, 
    # zoom_range=0.2, 
    # horizontal_flip=True, 
    # fill_mode='nearest'
)
valid_datagen = ImageDataGenerator(
    # rescale= 1./255, 
    # rotation_range=40, 
    # width_shift_range=0.2, 
    # height_shift_range= 0.2, 
    # shear_range= 0.2, 
    # zoom_range=0.2, 
    # horizontal_flip=True, 
    # fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_datagen.fit(X_train)

Alexnet

In [8]:
DefaultAlexConv2D = partial(keras.layers.Conv2D, strides=(4, 4) , activation = tf.nn.relu)
DefaultAlexMaxPool2D = partial(keras.layers.MaxPool2D, padding = 'SAME', data_format='channels_last')
DefaultBatchNormalize = partial(tf.keras.layers.BatchNormalization)
DefaultDropout = partial(tf.keras.layers.Dropout, rate = 0.5)

In [9]:
from datetime import datetime

date_time = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")

alex_net_model = keras.models.Sequential([
    DefaultAlexConv2D(filters = 96, kernel_size = (11, 11), input_shape = X_train.shape[1:], padding = 'VALID'),
    DefaultBatchNormalize(),
    DefaultAlexMaxPool2D(pool_size=(3, 3)), 
    DefaultAlexConv2D(filters = 256, kernel_size = (5, 5), padding = 'SAME'),
    DefaultBatchNormalize(),
    DefaultAlexMaxPool2D(pool_size=(3, 3)), 
    DefaultAlexConv2D(filters = 384, kernel_size = (3, 3),  padding = 'SAME'),
    DefaultBatchNormalize(),
    DefaultAlexConv2D(filters = 384, kernel_size = (3, 3),  padding = 'SAME'),
    DefaultBatchNormalize(),
    DefaultAlexConv2D(filters = 256, kernel_size = (3, 3),  padding = 'SAME'),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation=tf.nn.relu),
    DefaultDropout(),
    keras.layers.Dense(4096, activation=tf.nn.relu),
    DefaultDropout(),
    keras.layers.Dense(10, activation=tf.nn.softmax)
    ], name="alex_net_model_" + date_time)

In [10]:
alex_net_model.summary()

Model: "alex_net_model_02_01_2022_19_50_07"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 5, 96)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 256)         614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 2, 256)         1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (No

In [11]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,_,logs={}):
        if (logs.get("accuracy")==0.98 and logs.get("loss")<0.03):
            print("\nReached 98% accuracy so stopping training")
            self.model.stop_training =True
callbacks = myCallback()

In [12]:
alex_net_model.compile(loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'], optimizer=tf.optimizers.Adam())

history = alex_net_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, workers=10)
score = alex_net_model.evaluate(X_test, y_test)


Epoch 1/50
1407/1407 [==============================] - 13s 9ms/step - loss: 1.7415 - accuracy: 0.3683 - val_loss: 1.4347 - val_accuracy: 0.4876
Epoch 2/50
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4410 - accuracy: 0.4936 - val_loss: 1.4334 - val_accuracy: 0.4826
Epoch 3/50
1407/1407 [==============================] - 13s 9ms/step - loss: 1.2825 - accuracy: 0.5522 - val_loss: 1.4032 - val_accuracy: 0.4984
Epoch 4/50
1407/1407 [==============================] - 13s 9ms/step - loss: 1.1744 - accuracy: 0.5926 - val_loss: 1.3318 - val_accuracy: 0.5428
Epoch 5/50
1407/1407 [==============================] - 13s 9ms/step - loss: 1.0905 - accuracy: 0.6276 - val_loss: 1.2090 - val_accuracy: 0.5930
Epoch 6/50
1407/1407 [==============================] - 13s 9ms/step - loss: 1.0166 - accuracy: 0.6542 - val_loss: 1.0896 - val_accuracy: 0.6236
Epoch 7/50
1407/1407 [==============================] - 13s 9ms/step - loss: 0.9515 - accuracy: 0.6799 - val_loss: 1.0208 - val_ac

KeyboardInterrupt: 

In [ ]:

# history = alex_net_model.fit(train_datagen.flow(X_train, y_train, shuffle=True,  batch_size=BATCH_SIZE),
#                              validation_data = valid_datagen.flow(X_valid, y_valid, shuffle=True,  batch_size=BATCH_SIZE),
#                              use_multiprocessing = False, steps_per_epoch=STEPS_PER_EPOCH, epochs = 50, workers=20 , 
#                              callbacks= [callbacks])
# score = alex_net_model.evaluate(X_test, y_test, workers = 20)